$WX = y$ <br>

$ W $ = matriz de pesos <br>
$ X $ = matriz de embeddings <br>
$ Y $ = matriz de resultados <br>

En el presente trabajo se analizaron 3 algoritmos distintos con el objetivo de calcular los pesos de la matriz W. <br>

Se despejó la matriz $W$ calculando la Pseudo-Inversa de la matriz de embeddings $(X^{+})$, pero utilizando diferentes descomposiciones matriciales.

$W = YX^{+}$ <br>

In [69]:
import importlib
import alc
importlib.reload(alc)
import alcModulosNUMPY
importlib.reload(alcModulosNUMPY)

<module 'alcModulosNUMPY' from 'c:\\Users\\Sofía\\Desktop\\TP-ALC\\alcModulosNUMPY.py'>

In [44]:
from alc import cargarDataset

#### Lectura de Datos

In [ ]:
Xt, Yt, Xv, Yv = cargarDataset("template-alumnos/cats_and_dogs")

#### Algoritmo 2
Se busca la matriz de pesos W utilizando la descomposición en valores singulares para la resolución de la pseudoinversa. <br>

$X =U\Sigma V^{T}$ <br>

$X^{+} =V\Sigma^{+} U^{T}$<br>
$W = YX^{+}$





In [71]:
#Cálculo de la SVD de x CON NUMPY:
import numpy as np

U, S, Vt = np.linalg.svd(Xt)

n = Xt.shape[0]
m = Xt.shape[1]

Sigma = np.zeros((n, m))

# Colocar los valores singulares en la diagonal
# La cantidad de valores singulares es min(n, m)
for i in range(len(S)):
    Sigma[i, i] = S[i]



In [83]:
#Cálculo de la SVD de x:
from alcModulosNUMPY import svd_reducida

U, S, Vt = svd_reducida(Xt, k="max")

KeyboardInterrupt: 

In [ ]:
# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
from alc import pinvSVD

W_SVD = pinvSVD(U, Sigma, Vt, Yt)

### 6. Evaluación y Benchmarking
a. Para cada método de resolución de la W de los items anteriores, <br>
generar una matriz de confusión evaluando a partir de los pares de embeddings de validación o testing (Xv, Yv).

Algoritmo 2 - SVD

In [73]:
#Calculamos predicciones de Y
Y_pred = W_SVD @ Xv

clases_pred = np.argmax(Y_pred, axis=0)
clases_real = np.argmax(Yv, axis=0)

In [82]:
#Matriz de confusión
from sklearn.metrics import confusion_matrix

cmSVD = confusion_matrix(clases_real, clases_pred)

print(cmSVD)

[[334 166]
 [150 350]]


In [81]:
#Accuracy
from sklearn.metrics import accuracy_score

accsvd = accuracy_score(clases_real, clases_pred)

print("Accuracy de algoritmo 2 =",accsvd)

Accuracy de algoritmo 2 = 0.684


### 7. Síntesis Final